In [218]:
import numpy as np
from brainiak.isc import (isc, permutation_isc, bootstrap_isc,
                          phaseshift_isc, timeshift_isc,
                          compute_summary_statistic)

In [219]:
# Simulate correlated data by sampling from multivariate normal
def correlated_data(n_trs, n_subjects, r, mean=0, var=1):
    
    mean = np.full(n_subjects, mean)
    cov = np.full((n_subjects, n_subjects), r)
    np.fill_diagonal(cov, var)
    data = np.random.multivariate_normal(mean, cov, size=n_trs)
    
    return data

In [222]:
# Try a reasonable example with low N and decent correlation
n_trs = 300
n_subjects = 10
r = .3

# Generated our simulated correlated data
data = correlated_data(n_trs, n_subjects, r)

# Get leave-one-out ISC (we expect this to be inflated)
iscs = isc(data, pairwise=False)
mean_isc = compute_summary_statistic(iscs)
print(f"Leave-one-out ISC = {mean_isc}")

# Get pairwise ISC (not inflated)
iscs = isc(data, pairwise=True)
mean_isc = compute_summary_statistic(iscs)
print(f"Pairwise ISC = {mean_isc}")

Leave-one-out ISC = 0.4939674927637444
Pairwise ISC = 0.30562608853465517


In [240]:
# Function for running FPR simulations
def fpr_simulation(n_trs, n_subjects, test, r=.0, pairwise=False,
                   nominal_p = .05, n_simulations=1000, seed=None):

    np.random.seed(seed)
    p_distribution = []
    for i in np.arange(n_iterations):
        data = correlated_data(n_trs, n_subjects, r)

        if test == bootstrap_isc:
            iscs = isc(data, pairwise=pairwise)
            _, _, p, _ = test(iscs)

        elif test == permutation_isc:
            iscs = isc(data, pairwise=pairwise)
            _, p, _ = test(iscs)

        elif test in (phaseshift_isc, timeshift_isc):
            _, p, _ = test(data, pairwise=pairwise)

        if i > 0 and i % 10 == 0:
            print(f"Finished simulation {i}")

        p_distribution.append(p[0])
        
    p_distribution = np.array(p_distribution)
    fpr = np.sum(p_distribution <= nominal_p) / n_simulations
    
    return fpr, p_distribution

In [241]:
# Set common parameters across all noise simulations
n_trs = 300
n_subjects = 10
r = .0
pairwise = False
nominal_p = .05
n_simulations = 100

In [243]:
# Run the FPR simulation for the bootstrap test
test = bootstrap_isc
fpr, p_distribution = fpr_simulation(n_trs, n_subjects, test,
                                     n_simulations=100, seed=1)
print(f'FPR = {fpr:.3f} (for nominal p = {nominal_p}) using "{test.__name__}"')

Finished simulation 10
Finished simulation 20
Finished simulation 30
Finished simulation 40
Finished simulation 50
Finished simulation 60
Finished simulation 70
Finished simulation 80
Finished simulation 90
FPR = 0.160 (for nominal p = 0.05) using "bootstrap_isc"


In [244]:
# Run the FPR simulation for the permutation test
test = permutation_isc
fpr, p_distribution = fpr_simulation(n_trs, n_subjects, test,
                                     n_simulations=100, seed=1)
print(f'FPR = {fpr:.3f} (for nominal p = {nominal_p}) using "{test.__name__}"')

Finished simulation 10
Finished simulation 20
Finished simulation 30
Finished simulation 40
Finished simulation 50
Finished simulation 60
Finished simulation 70
Finished simulation 80
Finished simulation 90
FPR = 0.100 (for nominal p = 0.05) using "permutation_isc"


In [245]:
# Run the FPR simulation for the phase randomization test
test = phaseshift_isc
fpr, p_distribution = fpr_simulation(n_trs, n_subjects, test,
                                     n_simulations=100, seed=1)
print(f'FPR = {fpr:.3f} (for nominal p = {nominal_p}) using "{test.__name__}"')

Finished simulation 10
Finished simulation 20
Finished simulation 30
Finished simulation 40
Finished simulation 50
Finished simulation 60
Finished simulation 70
Finished simulation 80
Finished simulation 90
FPR = 0.130 (for nominal p = 0.05) using "phaseshift_isc"


In [246]:
# Run the FPR simulation for the circular time-shift test
test = timeshift_isc
fpr, p_distribution = fpr_simulation(n_trs, n_subjects, test,
                                     n_simulations=100, seed=1)
print(f'FPR = {fpr:.3f} (for nominal p = {nominal_p}) using "{test.__name__}"')

Finished simulation 10
Finished simulation 20
Finished simulation 30
Finished simulation 40
Finished simulation 50
Finished simulation 60
Finished simulation 70
Finished simulation 80
Finished simulation 90
FPR = 0.130 (for nominal p = 0.05) using "timeshift_isc"
